### The Basics of LangChain

In this notebook we'll explore exactly what LangChain is doing - and implement a straightforward example that lets us ask questions of a document!

First things first, let's get our dependencies all set!

In [1]:
pip install openai langchain -q

Note: you may need to restart the kernel to use updated packages.


You'll need to have an OpenAI API key for this next part - see [this](https://www.onmsft.com/how-to/how-to-get-an-openai-api-key/) if you haven't already set one up!

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

#### Helper Functions (run this cell)

In [3]:
from IPython.display import display, Markdown

def disp_markdown(text: str) -> None:
  display(Markdown(text))

### Our First LangChain ChatModel



---


<div class="warn">Note: Information on OpenAI's <a href=https://openai.com/pricing>pricing</a> and <a href=https://openai.com/policies/usage-policies>usage policies.</a></div>



---



Now that we're set-up with OpenAI's API - we can begin making our first ChatModel!

There's a few important things to consider when we're using LangChain's ChatModel that are outlined [here](https://python.langchain.com/en/latest/modules/models/chat.html)

Let's begin by initializing the model with OpenAI's `gpt-3.5-turbo` (ChatGPT) model.

We're not going to be leveraging the [streaming](https://python.langchain.com/en/latest/modules/models/chat/examples/streaming.html) capabilities in this Notebook - just the basics to get us started!

In [5]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")

If we look at the [Chat completions](https://platform.openai.com/docs/guides/chat) documentation for OpenAI's chat models - we'll see that there are a few specific fields we'll need to concern ourselves with:

`role`
- This refers to one of three "roles" that interact with the model in specific ways.
- The `system` role is an optional role that can be used to guide the model toward a specific task. Examples of `system` messages might be:
  - You are an expert in Python, please answer questions as though we were in a peer coding session.
  - You are the world's leading expert in stamps.

  These messages help us "prime" the model to be more aligned with our desired task!

- The `user` role represents, well, the user!
- The `assistant` role lets us act in the place of the model's outputs. We can (and will) leverage this for some few-shot prompt engineering!

Each of these roles has a class in LangChain to make it nice and easy for us to use!

Let's look at an example.

In [7]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# The SystemMessage is associated with the system role, setting the scene for an Astronomy context
system_message = SystemMessage(content="You are an astronomer at a space observatory.")

# The HumanMessage is associated with the user role, asking a question related to Astronomy
user_message = HumanMessage(content="Can you explain the significance of the Hubble Deep Field?")

# The AIMessage is associated with the assistant role, providing an informative response
assistant_message = AIMessage(content="Absolutely! The Hubble Deep Field is a groundbreaking image by the Hubble Space Telescope. It covers a small region in the constellation Ursa Major, depicting some of the youngest and most distant galaxies ever observed. This image has provided invaluable insights into the early universe, helping astronomers to understand galaxy formation and evolution.")


Now that we have those messages set-up, let's send them to `gpt-3.5-turbo` with a new user message and see how it does!

It's easy enough to do this - the ChatOpenAI model accepts a list of inputs!

In [11]:
second_user_message = HumanMessage(content="What about the LSST ?")

# create the list of prompts
list_of_prompts = [
    system_message,
    user_message,
    assistant_message,
    second_user_message
]

# we can just call our chat_model on the list of prompts!
chat_model.invoke(list_of_prompts)

AIMessage(content='The Large Synoptic Survey Telescope (LSST) is an upcoming ground-based telescope that will conduct a wide, fast, and deep survey of the entire southern sky. It will observe the night sky repeatedly over a ten-year period, creating a detailed map of the universe. The LSST is expected to revolutionize many areas of astronomy, including the study of dark matter and dark energy, the detection of asteroids, and the exploration of transient events such as supernovae. Its data will be made publicly available, allowing astronomers worldwide to access and study this wealth of information.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

Great! That's inline with what we expected to see!

### PromptTemplates

Next stop, we'll discuss a few templates. This allows us to easily interact with our model by not having to redo work we've already completed!

In [12]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# we can signify variables we want access to by wrapping them in {}
system_prompt_template = "You are an expert in {SUBJECT}, and you're currently feeling {MOOD}"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)

user_prompt_template = "{CONTENT}"
user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt_template)

# put them together into a ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])

Now that we have our `chat_prompt` set-up with the templates - let's see how we can easily format them with our content!

NOTE: `disp_markdown` is just a helper function to display the formatted markdown response.

In [14]:
# note the method `to_messages()`, that's what converts our formatted prompt into
formatted_chat_prompt = chat_prompt.format_prompt(SUBJECT="celestial objects", MOOD="curiously excited", CONTENT="Hi, what are the most fascinating celestial objects to observe in the night sky?").to_messages()

disp_markdown(chat_model.invoke(formatted_chat_prompt).content)

Hello! There are so many fascinating celestial objects to observe in the night sky, but some of the most popular and awe-inspiring ones include:

1. **The Moon**: Our closest celestial neighbor, the Moon offers a wealth of detail to observe, from its craters and seas to its changing phases.

2. **Planets**: Planets like Jupiter and Saturn are always popular targets for observation. Jupiter's cloud bands and four largest moons (Io, Europa, Ganymede, and Callisto) are especially fascinating, as are Saturn's iconic rings.

3. **Nebulae**: Nebulae are vast clouds of gas and dust where stars are born. The Orion Nebula (M42) is a popular target, known for its colorful gases and young stars.

4. **Galaxies**: The Andromeda Galaxy (M31) is a spectacular sight and the closest spiral galaxy to our own Milky Way. It's visible to the naked eye from dark skies and even more impressive through a telescope.

5. **Star Clusters**: Open clusters like the Pleiades (M45) and globular clusters like M13 in Hercules are beautiful groupings of stars that are great for observing with binoculars or a telescope.

6. **Meteor Showers**: While not individual objects, meteor showers can be incredibly exciting to observe. Events like the Perseids and Geminids can produce dozens of shooting stars per hour under dark skies.

7. **Comets**: Occasionally, a bright comet will grace the night sky, offering a stunning and rare sight. Comets like Hale-Bopp and NEOWISE have been memorable in recent years.

Each of these celestial objects offers a unique and captivating view of the universe, making stargazing a truly rewarding experience.

### Putting the Chain in LangChain

In essense, a chain is exactly as it sounds - it helps us chain actions together.

Let's take a look at an example.

In [18]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt)

disp_markdown(chain.run(SUBJECT="galaxies", MOOD="in awe", CONTENT="Is the Andromeda Galaxy on a collision course with the Milky Way?"))


Yes, the Andromeda Galaxy (M31) and the Milky Way are indeed on a collision course. Current scientific understanding suggests that the two galaxies are approaching each other at a speed of about 110 kilometers per second and are expected to collide in about 4.5 billion years. This collision will result in the formation of a new galaxy, often referred to as Milkomeda or Milkdromeda. The collision will be a spectacular event in cosmic terms and will likely reshape both galaxies as they merge and interact gravitationally. It's a truly awe-inspiring and humbling aspect of the vastness and dynamics of the universe.

### Index Local Files

Now that we've got our first chain running, let's talk about indexing and what we can do with it!

For the purposes of this tutorial, we'll be using the word "index" to refer to a collection of documents organized in a way that is easy for LangChain to access them as a "Retriever".

Let's check out the Retriever set-up! First, a new dependency!

In [ ]:
!pip install chromadb tiktoken nltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject

In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/julien/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Before we can get started with our chain - we'll have to include some kind of text that we want to include as potential context.

Let's use Douglas Adam's [The Hitch Hiker's Guide to the Galaxy](https://erki.lap.ee/failid/raamatud/guide1.txt) as our text file.

In [20]:
%pwd

'/home/julien/code/JulsdL/huggingface_nlp'

In [21]:
!wget https://erki.lap.ee/failid/raamatud/guide1.txt

--2024-03-20 02:32:41--  https://erki.lap.ee/failid/raamatud/guide1.txt
Resolving erki.lap.ee (erki.lap.ee)... 185.158.177.102
Connecting to erki.lap.ee (erki.lap.ee)|185.158.177.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291862 (285K) [text/plain]
Saving to: ‘guide1.txt’

guide1.txt          100%[===================>] 285.02K  1.09MB/s    in 0.3s    

2024-03-20 02:32:42 (1.09 MB/s) - ‘guide1.txt’ saved [291862/291862]



In [22]:
from langchain.document_loaders import TextLoader
loader = TextLoader('guide1.txt', encoding='utf8')

Now we can set up our first Index!

More detail can be found [here](https://python.langchain.com/en/latest/modules/indexes/getting_started.html) but we'll skip to a more functional implementation!

In [23]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

/home/julien/.pyenv/versions/3.11.3/envs/huggingface/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Now that we have our Index set-up, we can query it straight away!

In [27]:
query = "What is the significance of the number 42 in 'The Hitchhiker's Guide to the Galaxy'?"
index.query_with_sources(query)

{'question': "What is the significance of the number 42 in 'The Hitchhiker's Guide to the Galaxy'?",
 'answer': " The number 42 does not have any significance in 'The Hitchhiker's Guide to the Galaxy'.\n",
 'sources': ''}

### Putting it All Together

Now that we have a simple idea of how we prompt, what a chain is, and has some local data - let's put it all together!

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [30]:
with open("guide1.txt") as f:
    hitchhikersguide = f.read()

Next we'll want to split our text into appropirately sized chunks.

We're going to be using the NLTKTextSplitter from LangChain today.

The size of these chunks will depend heavily on a number of factors relating to which LLM you're using, what the max context size is, and more.

You can also choose to have the chunks overlap to avoid potentially missing any important information between chunks. As we're dealing with a novel - there's not a critical need to include overlap.

We can also pass in the separator - this is what we'll try and separate the documents on. Be careful to understand your documents so you can be sure you use a valid separator!

For now, we'll go with 1000 characters.

In [31]:
from langchain.text_splitter import NLTKTextSplitter
text_splitter = NLTKTextSplitter()
texts = text_splitter.split_text(hitchhikersguide)

Now that we've split our document into more manageable sized chunks. We'll need to embed those documents!

For more information on embedding - please check out this resource from OpenAI.

In order to do this, we'll first need to select a method to embed - for this example we'll be using OpenAI's embedding - but you're free to use whatever you'd like.

You just need to ensure you're using consistent embeddings as they don't play well with others.

In [32]:
embeddings = OpenAIEmbeddings()


Now that we've set up how we want to embed our document - we'll need to embed it.

For this week we'll be glossing over the technical details of this process - as we'll get more into next week.

Just know that we're converting our text into an easily queryable format!

In [33]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Finally, we're able to combine what we've done so far into a chain!

We're going to leverage the load_qa_chain to quickly integrate our queryable documents with an LLM.

There are 4 major methods of building this chain, they can be found here!

For this example we'll be using the stuff chain type.

In [45]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine")
query = "What is the space ship maximum velocity ?"
docs = docsearch.get_relevant_documents(query)
chain.invoke({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nThe maximum velocity of the space ship is not explicitly stated in the given context, but based on the description of the space ship being able to travel at R17 and above, it can be assumed that the maximum velocity is extremely high. However, it is also mentioned that the velocity can vary depending on the awareness of the third factor, and if not handled with tranquility, it can result in stress, ulcers, and even death. Additionally, in the given context, it is stated that the aircar rocketed them at speeds in excess of R17, indicating that the maximum velocity of the space ship could potentially be higher than R17. However, the exact maximum velocity of the space ship is still unknown and can vary depending on the circumstances.'}

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)